# Data from comet_ml import ExperimentPreparation

In [1]:
from comet_ml import Experiment
import sys
sys.path.append('../ift6758/data/')
from question_2_m2 import *
from question_4_m2 import get_new_features, get_angle_change
import numpy as np
import json
import time
import json
import pandas as pd
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
#df = tidy_df_loop([2016, 2017, 2018, 2019])

In [3]:
#df.to_csv('tidy_df2.csv')

In [4]:
df = pd.read_csv('tidy_df2.csv')
df = df.drop(columns = 'Unnamed: 0')
df = df[df['season_type'] == 'R']
df = df.reset_index()

Add features from q2 and q4

In [5]:
list_angle = []
for i in range(0,df.shape[0]):

    if df.attacking_team_side[i] == "right":
        if df.y_coordinates[i] == 0:
            list_angle.append(0)
        elif df.y_coordinates[i] > 0:
            list_angle.append(np.arcsin(df.y_coordinates[i]/df.distance_from_net[i])*-180/math.pi)
        else:
            list_angle.append(np.arcsin(df.y_coordinates[i]/df.distance_from_net[i])*-180/math.pi)
            
    elif df.attacking_team_side[i] == "left":
        if df.y_coordinates[i] == 0:
            list_angle.append(0)
        elif df.y_coordinates[i] > 0:
            list_angle.append(np.arcsin(df.y_coordinates[i]/df.distance_from_net[i])*180/math.pi)
        else:
            list_angle.append(np.arcsin(df.y_coordinates[i]/df.distance_from_net[i])*180/math.pi)
df['angle_from_net'] = list_angle

df = get_new_features(df)
df = get_angle_change(df)

Add 'defending_team' feature, one hot encode

In [6]:
df['defending_team'] = np.where(df['attacking_team'] == df['home_team'], df['away_team'], df['home_team']) + "_def"
df = df.join(pd.get_dummies(df[['defending_team', 'attacking_team']]))

#cols_to_drop = [s for s in list(df.columns) if 'goalie' in s]
#df = df.drop(columns=cols_to_drop)

bit of clean up

In [7]:
# keep only numeric columns
df = df.select_dtypes(include=np.number)

# drop some irrelevant columns
cols_to_drop = ['game_id', 'event_id', 'shot_ind', 'x_coordinates', 'y_coordinates', 
                'previous_event_x_coordinates', 'previous_event_y_coordinates', 'previous_event_period', 
                'home_players', 'away_players', 'previous_event_game_seconds', 'previous_event_game_seconds']
df = df.drop(columns = cols_to_drop)

# impute NAs
df['distance_from_net'] = df['distance_from_net'].fillna(df['distance_from_net'].median())
df['distance_from_last_event'] = df['distance_from_last_event'].fillna(df['distance_from_last_event'].median())
df['speed'] = df['speed'].fillna(df['speed'].median())
df['speed'] = df['speed'].replace([np.inf, -np.inf], df['speed'].median())
df['angle_from_net'] = df['angle_from_net'].fillna(df['angle_from_net'].median())
df['change_in_angle'] = df['change_in_angle'].fillna(df['change_in_angle'].median())

Create train and validation sets, convert df to torch tensor

In [8]:
df_train = df[df['season'] != 20182019]
df_valid = df[df['season'] == 20182019]

X_train = df_train.drop(columns = ['season', 'goal_ind'])
y_train = df_train['goal_ind']

X_valid = df_valid.drop(columns = ['season', 'goal_ind'])
y_valid = df_valid['goal_ind']

In [9]:
X_train = torch.tensor(X_train.to_numpy()).float()
X_valid = torch.tensor(X_valid.to_numpy()).float()

y_train = torch.tensor(y_train.to_numpy())
y_valid = torch.tensor(y_valid.to_numpy())

## 2 layer Neural Net Softmax Classifier

In [ ]:
jj=1
for hidden_dim in [8, 16, 32]:
    for lr in [0.0002, 0.001, 0.005]:

        # model class
        class binary_classifier(nn.Module):

            def __init__(self):
                super(binary_classifier,self).__init__()
                self.layer1 = nn.Linear(X_train.shape[1], hidden_dim)
                self.layer2 = nn.Linear(hidden_dim, 2)

            def forward(self,x):
                x = F.relu(self.layer1(x))
                x = self.layer2(x)
                return x

            #This function takes an input and predicts the class, (0 or 1)        
            def predict_class(self,x):
                #Apply softmax to output. 
                pred = F.softmax(self.forward(x), dim=1)
                ans = []
                #Pick the class with maximum weight
                for t in pred:
                    if t[0]>t[1]:
                        ans.append(0)
                    else:
                        ans.append(1)
                return torch.tensor(ans)

            def predict_probs(self,x):
                #Apply softmax to output. 
                pred = F.softmax(self.forward(x), dim=1)
                return pred
            
        # initialise model
        model = binary_classifier()
        loss_computation = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        
        # trainning
        epochs = 2000
        
        # Store metrics
        epochs_list = []
        losses = []
        losses_valid = []
        train_rocs = []
        valid_rocs = []

        for i in range(epochs):
            y_pred = model.forward(X_train)
            loss = loss_computation(y_pred,y_train)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if (i+1) % 100 == 0:
                preds = model.predict_probs(X_train)
                goal_index = np.argmin(preds.sum(dim=0).detach().numpy())
                fpr, tpr, _ = roc_curve(np.array(y_train), preds[:,goal_index].detach().numpy())
                roc_auc = auc(fpr, tpr)

                preds_valid = model.predict_probs(X_valid)
                goal_index_valid = np.argmin(preds_valid.sum(dim=0).detach().numpy())
                fpr_valid, tpr_valid, _ = roc_curve(np.array(y_valid), preds_valid[:,goal_index_valid].detach().numpy())
                roc_auc_valid = auc(fpr_valid, tpr_valid)

                y_pred_valid = model.forward(X_valid)
                loss_valid = loss_computation(y_pred_valid, y_valid)

                epochs_list.append(i+1)
                losses.append(loss.item())
                losses_valid.append(loss_valid.item())
                train_rocs.append(roc_auc)
                valid_rocs.append(roc_auc_valid)
            
        # plot metrics
        fig, axs = plt.subplots(1, 2, figsize=(16, 4))
        fig.suptitle(f'hidden dim = {hidden_dim}, learning rate = {lr}')
        axs[0].plot(epochs_list, losses)
        axs[0].plot(epochs_list, losses_valid)
        axs[0].set_title(f'Cross Entropy Loss, best valid score = {min(losses_valid)}', fontsize=11)
        axs[0].legend(labels = ['train', 'valid'])
        axs[0].set_ylabel('Loss')
        axs[0].set_xlabel('Training Epoch')
        axs[0].set_ylim([0, 1.05*max([max(losses), max(losses_valid)])])
        
        axs[1].plot(epochs_list, train_rocs)
        axs[1].plot(epochs_list, valid_rocs)
        axs[1].set_title(f'ROC AUC, best valid score = {max(valid_rocs)}', fontsize=11)
        axs[1].legend(labels = ['train', 'valid'])
        axs[1].set_ylabel('AUC')
        axs[1].set_xlabel('Training Epoch')
        axs[1].set_ylim([ 0.95*min([min(train_rocs), min(valid_rocs)]), 1.05*max([max(train_rocs), max(valid_rocs)])])
        print(jj)
        jj += 1

1


## 3 layer Neural Net softmax classifier

In [ ]:
jj=1
for hidden_dims in [[32,8], [16,16]]:
    for lr in [0.0002, 0.001, 0.005]:

        # model class
        class binary_classifier(nn.Module):

            def __init__(self):
                super(binary_classifier,self).__init__()
                self.layer1 = nn.Linear(X_train.shape[1], hidden_dims[0])
                self.layer2 = nn.Linear(hidden_dims[0], hidden_dims[1])
                self.layer3 = nn.Linear(hidden_dims[1], 2)

            def forward(self,x):
                x = F.relu(self.layer1(x))
                x = F.relu(self.layer2(x))
                x = self.layer3(x)
                return x

            #This function takes an input and predicts the class, (0 or 1)        
            def predict_class(self,x):
                #Apply softmax to output. 
                pred = F.softmax(self.forward(x), dim=1)
                ans = []
                #Pick the class with maximum weight
                for t in pred:
                    if t[0]>t[1]:
                        ans.append(0)
                    else:
                        ans.append(1)
                return torch.tensor(ans)

            def predict_probs(self,x):
                #Apply softmax to output. 
                pred = F.softmax(self.forward(x), dim=1)
                return pred
            
        # initialise model
        model = binary_classifier()
        loss_computation = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        
        # trainning
        epochs = 2000
        
        # Store metrics
        epochs_list = []
        losses = []
        losses_valid = []
        train_rocs = []
        valid_rocs = []

        for i in range(epochs):
            y_pred = model.forward(X_train)
            loss = loss_computation(y_pred,y_train)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if (i+1) % 100 == 0:
                preds = model.predict_probs(X_train)
                goal_index = np.argmin(preds.sum(dim=0).detach().numpy())
                fpr, tpr, _ = roc_curve(np.array(y_train), preds[:,goal_index].detach().numpy())
                roc_auc = auc(fpr, tpr)

                preds_valid = model.predict_probs(X_valid)
                goal_index_valid = np.argmin(preds_valid.sum(dim=0).detach().numpy())
                fpr_valid, tpr_valid, _ = roc_curve(np.array(y_valid), preds_valid[:,goal_index_valid].detach().numpy())
                roc_auc_valid = auc(fpr_valid, tpr_valid)

                y_pred_valid = model.forward(X_valid)
                loss_valid = loss_computation(y_pred_valid, y_valid)

                epochs_list.append(i+1)
                losses.append(loss.item())
                losses_valid.append(loss_valid.item())
                train_rocs.append(roc_auc)
                valid_rocs.append(roc_auc_valid)
            
        # plot metrics
        fig, axs = plt.subplots(1, 2, figsize=(16, 4))
        fig.suptitle(f'hidden dim = {hidden_dim}, learning rate = {lr}')
        axs[0].plot(epochs_list, losses)
        axs[0].plot(epochs_list, losses_valid)
        axs[0].set_title(f'Cross Entropy Loss, best valid score = {min(losses_valid)}', fontsize=11)
        axs[0].legend(labels = ['train', 'valid'])
        axs[0].set_ylabel('Loss')
        axs[0].set_xlabel('Training Epoch')
        axs[0].set_ylim([0, 1.05*max([max(losses), max(losses_valid)])])
        
        axs[1].plot(epochs_list, train_rocs)
        axs[1].plot(epochs_list, valid_rocs)
        axs[1].set_title(f'ROC AUC, best valid score = {max(valid_rocs)}', fontsize=11)
        axs[1].legend(labels = ['train', 'valid'])
        axs[1].set_ylabel('AUC')
        axs[1].set_xlabel('Training Epoch')
        axs[1].set_ylim([ 0.95*min([min(train_rocs), min(valid_rocs)]), 1.05*max([max(train_rocs), max(valid_rocs)])])
        
        print(jj)
        jj += 1

## Retrain final model with best hyperparams, log into comet, save model

In [ ]:
torch.save(model.state_dict(), PATH)

model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.eval()